In [11]:
from GlassBox.ebm.ebm_utils import EBMGridSearch
from interpret.glassbox import ExplainableBoostingRegressor as EBRegressor
import numpy as np
import pandas as pd
%reload_ext autoreload
%autoreload 2

# Grid-search

In [12]:
EBM_parameter = dict(
    learning_rate= [0.01, 0.02, 0.03],
    min_samples_leaf=[2, 3, 4, 5, 6, 7, 8, 9, 10],
    max_leaves=[2, 3, 4, 5, 6, 7],
)

gs = EBMGridSearch(path_dataset="../../outputs/scores.csv",
                   task="Regressor", random_state=841)
best_ = gs.grid_search(EBMModel=EBRegressor, hyperparameters=EBM_parameter)
gs.save_model_parameters(best_[0],best_[1], name="model_regressor")

# Metric evaluation

In [14]:
best_model, parameters = gs.load_model_parameters(name="model_regressor")
best_model,parameters

In [15]:
# nDCG on test-set
nDCG_train = gs.eval_model(model=best_model, df=gs.train, nDCG_at=[1,10,15])
nDCG_valid = gs.eval_model(model=best_model, df=gs.valid, nDCG_at=[1,10,15])
nDCG_test = gs.eval_model(model=best_model, df=gs.test, nDCG_at=[1,10,15])

display(pd.DataFrame([nDCG_train,nDCG_valid,nDCG_test],index=["Training","Validation","Test"]))

nDCG@15
Training set 0.7383
Validation set 0.7712
Test set 0.7579


## Example of Job-offer

In [16]:
qId = 1
job_curricula = gs.test[gs.test["qId"] == qId]

y_pred = best_model.predict(np.asarray(job_curricula.iloc[:, 2:13]))

y_pred = pd.DataFrame(y_pred, index=job_curricula.index, columns=["lambdas"])
dt_final = pd.merge(job_curricula, y_pred, left_index=True, right_index=True)

In [17]:
dt_final.sort_values("labels", ascending=False)["labels"].head(15)

990     4
1274    4
915     4
1371    4
1160    4
1373    4
1401    4
906     3
1249    3
1456    3
1086    3
1416    3
1450    3
853     3
887     3
Name: labels, dtype: int64

In [18]:
dt_final.sort_values("lambdas",ascending=False)["labels"].head(15)

968     3
1086    3
876     3
1070    3
1271    3
1409    2
1018    1
1442    2
1294    3
962     2
997     3
1165    2
1121    2
946     2
990     4
Name: labels, dtype: int64